<a href="https://colab.research.google.com/github/Dadoyen2/Geometry-Optimization/blob/main/geometry_optimization_in_progress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import requests
import math
def get_coordinate(url):
    atoms = []
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.splitlines()

    # Start parsing for atom data directly after identifying the pattern
    for line in lines:
        parts = line.split()
        # Check if line contains a potential atom data pattern
        if len(parts) >= 4 and parts[3].isalpha():
            try:
                # Attempt to parse the first three values as coordinates
                x, y, z = map(float, parts[:3])
                atom_type = parts[3]
                atoms.append((atom_type, x, y, z))
            except ValueError:
                # Skip lines that don't fit the pattern (ensures generality)
                continue

    return atoms

def print_atom_coordinates(atoms):
    print(f"The input file has: {len(atoms)} atoms")
    print("Atoms and coordinates (in Å):")
    for atom in atoms:
        print(f"{atom[0]:<2} {atom[1]:>8.4f} {atom[2]:>8.4f} {atom[3]:>8.4f}")

# Example URL to use with the general extraction function
url = 'https://raw.githubusercontent.com/Dadoyen2/Geometry-Optimization/main/ethane_dist.mol2'
atoms = get_coordinate(url)
print_atom_coordinates(atoms)




The input file has: 8 atoms
Atoms and coordinates (in Å):
C   -0.7560   0.0500   0.0000
C    0.7960   0.0000   0.0500
H   -1.1804   0.6486   0.7945
H   -1.1004   0.3601  -0.9626
H   -1.2405  -0.9987   0.4781
H    1.1204  -0.3401   0.9526
H    1.1605   1.0187  -0.1581
H    1.1904  -0.6786  -0.7745


In [47]:
import numpy as np
import requests

def bond_length(coord1, coord2):
    return np.linalg.norm(np.array(coord2) - np.array(coord1))

def bond_angle(coord1, coord2, coord3):
    vec1 = np.array(coord1) - np.array(coord2)
    vec2 = np.array(coord3) - np.array(coord2)
    cos_theta = np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))
    return np.degrees(np.arccos(cos_theta))



def dihedral_angle(coord1, coord2, coord3, coord4):
    # Define the vectors
    vec1 = np.array(coord1) - np.array(coord2)  # r_AB
    vec2 = np.array(coord3) - np.array(coord2)  # r_BC
    vec3 = np.array(coord4) - np.array(coord3)  # r_CD

    # Calculate normal vectors to planes
    t = np.cross(vec1, vec2)  # Normal to plane ABC
    u = np.cross(vec2, vec3)  # Normal to plane BCD

    # Calculate vector perpendicular to both normal vectors
    v = np.cross(t, u)

    # Compute cos(phi)
    cos_phi = np.dot(t, u) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Compute sin(phi)
    sin_phi = np.dot(vec2 / np.linalg.norm(vec2), v) / (np.linalg.norm(t) * np.linalg.norm(u))

    # Correct numerical inaccuracies (clipping for cos_phi)
    cos_phi = np.clip(cos_phi, -1.0, 1.0)

    # Calculate dihedral angle
    angle_rad = np.arctan2(sin_phi, cos_phi)  # Angle in radians
    angle_degree = np.degrees(angle_rad)     # Convert radians to degrees

    # Normalize angle to the range 0° to 360°
    if angle_degree < 0:
        angle_degree += 360
    angle_rad = np.radians(angle_degree)

  #  return angle_degree, angle_rad

    # Subtract the angle from 180 degrees
    corrected_angle_degree = angle_degree -180
    corrected_angle_rad = np.radians(corrected_angle_degree)

    return corrected_angle_degree, corrected_angle_rad


# Calculate potential energy and count parameters
def calculate_energy(atoms, bond_params, angle_params, dihedral_params, lj_params):
    bond_count, angle_count, dihedral_count = 0, 0, 0
    stretch_energy, bend_energy, torsion_energy, vdw_energy = 0.0, 0.0, 0.0, 0.0

# Stretching (Bond) Energy Calculation
    # Parameters for bond lengths and constants
bond_params = {
    ('C', 'H'): (350, 1.11),  # Example: k_b and r_0 for C-H bond
    ('C', 'C'): (300, 1.53),  # Example: k_b and r_0 for C-C bond
    #
}

# Define distance cutoffs to distinguish bonded vs non-bonded interactions
bond_cutoffs = {
    ('C', 'H'): (0.5, 1.5),  # Minimum and maximum cutoff for C-H bond detection
    ('C', 'C'): (1.0, 2.0),  # Minimum and maximum cutoff for C-C bond detection
    # Add other cutoffs as needed
}

# Stretching (Bond) Energy Calculation with flexibility for any C-H or hydrocarbon bonds
stretch_energy = 0.0
bond_count = 0
visited_bonds = set()  # To keep track of counted bonds
bonds_new = []
for i in range(len(atoms) - 1):
    for j in range(i + 1, len(atoms)):
        bond_type = tuple(sorted((atoms[i][0], atoms[j][0])))  # Determine bond type dynamically

        # Only consider bond types defined in bond_params and bond_cutoffs
        if bond_type in bond_params and bond_type in bond_cutoffs:
            # Calculate bond length
            r = bond_length(atoms[i][1:], atoms[j][1:])

            # Apply distance cutoff to avoid counting non-bonded pairs
            min_cutoff, max_cutoff = bond_cutoffs[bond_type]
            if min_cutoff <= r <= max_cutoff and (i, j) not in visited_bonds:
                visited_bonds.add((i, j))  # Mark this bond as visited

                # Get the bond parameters
                k_b, r_0 = bond_params[bond_type]
                bonds_new.append([i,j])
                # Calculate energy for this bond and add to total
                bond_energy = k_b * (r - r_0) ** 2
                stretch_energy += bond_energy
                bond_count += 1

                # Debugging output for verification
                print(f"Bond {bond_type[0]}-{bond_type[1]}: Length = {r:.2f} Å, Energy = {bond_energy:.4f} kcal/mol")

# Output the final results
print(f"\nTotal Stretching (Bond) Energy: {stretch_energy:.4f} kcal/mol")
print(f"Number of Bonds: {bond_count}")




Bond C-C: Length = 1.55 Å, Energy = 0.1672 kcal/mol
Bond C-H: Length = 1.08 Å, Energy = 0.2841 kcal/mol
Bond C-H: Length = 1.07 Å, Energy = 0.6071 kcal/mol
Bond C-H: Length = 1.25 Å, Energy = 6.8832 kcal/mol
Bond C-H: Length = 1.02 Å, Energy = 2.9857 kcal/mol
Bond C-H: Length = 1.10 Å, Energy = 0.0237 kcal/mol
Bond C-H: Length = 1.14 Å, Energy = 0.2814 kcal/mol

Total Stretching (Bond) Energy: 11.2323 kcal/mol
Number of Bonds: 7


In [48]:
import math

# Define parameters for geometry optimization
bond_distance_threshold = 1.6 # Threshold distance to consider atoms as bonded (in angstroms)
equilibrium_angle_degrees = 109.5  # Equilibrium angle in degrees for sp³ carbons
equilibrium_angle_radians = math.radians(equilibrium_angle_degrees)  # Convert to radians

# Define k_a values in kcal mol^-1 degree^-2 and their conversions to radians^-2
ka_values_degree = {
    ('H', 'C', 'H'): 35,  # kcal mol^-1 degree^-2
    ('H', 'C', 'C'): 35,  # kcal mol^-1 degree^-2
    ('C', 'C', 'C'): 60   # kcal mol^-1 degree^-2
}
ka_values_radian = {angle_type: k * (math.pi / 180) ** 2 for angle_type, k in ka_values_degree.items()}

# Initialize variables for bending energy calculation
bend_energy_radians = 0.0
bend_energy_degrees = 0.0
angle_count = 0
angles_new = []
for j in range(len(atoms)):
    # Only consider carbon atoms as central atoms for angle calculations
    if atoms[j][0] == 'C':
        # Identify atoms directly bonded to this central carbon atom 'j'
        bonded_atoms = [
            i for i in range(len(atoms))
            if i != j and bond_length(atoms[i][1:], atoms[j][1:]) <= bond_distance_threshold
        ]

        # Iterate over all unique pairs of bonded atoms to calculate angles

        for m in range(len(bonded_atoms)):
            for n in range(m + 1, len(bonded_atoms)):
                i, k = bonded_atoms[m], bonded_atoms[n]

                # Define angle type and get the appropriate k_a values
                angle_type = (atoms[i][0], atoms[j][0], atoms[k][0])
                ka_degree = ka_values_degree.get(angle_type, 35)  # Default to 35 if not found
                ka_radian = ka_values_radian.get(angle_type, 35 * (math.pi / 180) ** 2)  # Default conversion
                #print(f"Using k_a values for angle type {angle_type}: ka_degree = {ka_degree:.3f} kcal/mol/degree^2, ka_radian = {ka_radian:.5f} kcal/mol/radian^2")
                # Calculate the angle in degrees and convert it to radians
                angle_degrees = bond_angle(atoms[i][1:], atoms[j][1:], atoms[k][1:])
                angle_radians = math.radians(angle_degrees)

                # Format angles to 3 decimal places
                #angle_degrees = float(f"{angle_degrees:.3f}")
                #angle_radians = float(f"{angle_radians:.3f}")

                # Calculate the bending energy for this angle in both radians and degrees
                # Energy in radians
                angle_energy_radians = ka_radian * (angle_radians - equilibrium_angle_radians) ** 2
                bend_energy_radians += angle_energy_radians

                # Energy in degrees
                angle_energy_degrees = ka_degree * (angle_degrees - equilibrium_angle_degrees) ** 2
                bend_energy_degrees += angle_energy_degrees

                angle_count += 1

                angles_new.append([i,j,k])

                # Print the angle in both degrees and radians and its corresponding energy
                print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1}: "
                      f"{angle_degrees:>10.3f} {angle_radians: >10.3f} {angle_energy_degrees:>20.6f}  {angle_energy_radians:>20.6f}")

# Optionally, print the total bending energy and angle count at the end
print(f"\nTotal bending energy in radians: {bend_energy_radians:.4f} kcal/mol")
print(f"Total bending energy in degrees: {bend_energy_degrees:.4f} kcal/mol")
print(f"Total number of angles calculated: {angle_count}")



C2 - C1 - H3:    112.717      1.967           362.187201              0.000034
C2 - C1 - H4:    111.123      1.939            92.193260              0.000009
C2 - C1 - H5:    110.354      1.926            25.538209              0.000002
H3 - C1 - H4:    112.009      1.955           220.283660              0.000020
H3 - C1 - H5:     91.792      1.602         10975.255770              0.001018
H4 - C1 - H5:    117.587      2.052          2289.193178              0.000212
C1 - C2 - H6:    110.962      1.937            74.801757              0.000007
C1 - C2 - H7:    107.137      1.870           195.496814              0.000018
C1 - C2 - H8:    109.998      1.920             8.667359              0.000001
H6 - C2 - H7:    111.782      1.951           182.190328              0.000017
H6 - C2 - H8:    109.435      1.910             0.146832              0.000000
H7 - C2 - H8:    107.443      1.875           148.139085              0.000014

Total bending energy in radians: 0.0014 kcal/mol
To

In [49]:
from re import X
import math
import numpy as np
import requests

# Torsion energy parameters
torsion_params = {
    "v_n": 0.3,  # kcal/mol, torsional barrier
    "n": 3,      # Periodicity of torsion angle (for typical organic molecules like alkanes)
                     # Equilibrium torsion angle in degrees (staggered conformation)
}


# Fixed bond length threshold for bonding (in Å, assuming a general threshold suitable for typical C-H and C-C bonds)
bond_thresh = 1.6  # Adjust this based on typical bond lengths in the molecule type
# Function to calculate dihedral (torsion) angle between four points in 3D
import numpy as np



# Function to create bond graph based on fixed distance threshold
def connectivity(atoms):
    coords = [atom[1:] for atom in atoms]  # Extract only the coordinates for each atom
    n_atoms = len(atoms)
    connectivity = [[] for _ in range(n_atoms)]
    for i in range(n_atoms):
        for j in range(i + 1, n_atoms):
            if bond_length(coords[i], coords[j]) < bond_thresh:
                connectivity[i].append(j)
                connectivity[j].append(i)

    return connectivity



# Function to find all possible torsion angles in the molecule
def get_torsions(atoms, connectivity):
    coords = [atom[1:] for atom in atoms]
    n_atoms = len(atoms)
    torsions = []
    for j in range(n_atoms):
        for a in range(len(connectivity[j])):
            k = connectivity[j][a]
            if k < j:
                continue
            for b in range(len(connectivity[j])):
                i = connectivity[j][b]
                if i == k:
                    continue
                for c in range(len(connectivity[k])):
                    l = connectivity[k][c]
                    if l == j or l == i:
                        continue
                    angle_deg, angle_rad = dihedral_angle(coords[i], coords[j], coords[k], coords[l])
                    torsions.append((i, j, k, l, angle_deg, angle_rad))
    return torsions

def calculate_torsion_energy(torsion_angle, in_degrees=True):
    if in_degrees:
        # Convert angle from degrees to radians if necessary
        torsion_angle = np.radians(torsion_angle)
    #phi0_rad = np.radians(torsion_params["phi0"])
    return torsion_params["v_n"] * (1 + np.cos(torsion_params["n"] * torsion_angle))
# Update the print statement in the for loop


def calculate_and_print_torsions(atoms):
    bond_graph = connectivity(atoms)
    torsions = get_torsions(atoms, bond_graph)

    torsion_energy_total_deg = 0.0
    torsion_energy_total_rad = 0.0
    print("\nList of all torsion angles with energies:")

    for (i, j, k, l, angle_deg, angle_rad) in torsions:
        # Calculate energies in degrees and radians
        torsion_energy_deg = calculate_torsion_energy(angle_deg, in_degrees=True)
        torsion_energy_rad = calculate_torsion_energy(angle_rad, in_degrees=False)

        # Accumulate total energies
        torsion_energy_total_deg += torsion_energy_deg
        torsion_energy_total_rad += torsion_energy_rad

        # Print each torsion with both atom type and index
        print(f"{atoms[i][0]}{i + 1} - {atoms[j][0]}{j + 1} - {atoms[k][0]}{k + 1} - {atoms[l][0]}{l + 1}: "
              f"{angle_deg:>10.3f} {angle_rad: >10.3f} {torsion_energy_deg:>20.6f}")


#print(f"Torsion: Atom {str(atoms[i][0])}-{i}-{str(atoms[j][0])}-{j}-{str(atoms[k][0])}-{k}-{str(atoms[l][0])}-{l}. "
 #     f"Angle: {angle_deg:.3f} degrees, {angle_rad:.4f} radians, "
  #    f"Energy (degrees): {torsion_energy_deg:.5f} kcal/mol, Energy (radians): {torsion_energy_rad:.5f} kcal/mol")



    # Print the total torsion energy in both degrees and radians
    print(f"\nTotal torsion energy (calculated using degrees): {torsion_energy_total_deg:.4f} kcal/mol")
    print(f"Total torsion energy (calculated using radians): {torsion_energy_total_rad:.4f} kcal/mol")
    print(f"Number of torsions calculated: {len(torsions)}")

# Example URL and execution
#url = 'https://raw.githubusercontent.com/Dadoyen2/Geometry-Optimization/main/nbutane.mol2'
#atoms = extract_atom_coordinates_from_url(url)
#print_atom_coordinates(atoms)
calculate_and_print_torsions(atoms)










List of all torsion angles with energies:
H3 - C1 - C2 - H6:     56.162      0.980             0.006038
H3 - C1 - C2 - H7:    -66.111     -1.154             0.015228
H3 - C1 - C2 - H8:    177.400      3.096             0.002775
H4 - C1 - C2 - H6:   -177.164     -3.092             0.003302
H4 - C1 - C2 - H7:     60.563      1.057             0.000130
H4 - C1 - C2 - H8:    -55.925     -0.976             0.006802
H5 - C1 - C2 - H6:    -44.877     -0.783             0.089239
H5 - C1 - C2 - H7:   -167.150     -2.917             0.065381
H5 - C1 - C2 - H8:     76.362      1.333             0.103519

Total torsion energy (calculated using degrees): 0.2924 kcal/mol
Total torsion energy (calculated using radians): 0.2924 kcal/mol
Number of torsions calculated: 9


In [50]:
# Function to calculate van der Waals energy and print as it iterates
def calculate_and_print_vdw_energy(atoms, bonds_new, angles_new):
    # Define epsilon and sigma values for Hydrogen and Carbon
    epsilon_values = {'H': 0.03, 'C': 0.07}  # in kcal/mol
    sigma_values = {'H': 1.20, 'C': 1.75}    # in Angstroms

    total_energy = 0.0

    print(f"{'Atom Pair':<15} {'Distance (Å)':>15} {'Energy (kcal/mol)':>20}")

    N = len(atoms)

    # Iterate over all unique atom pairs
    for i in range(N):
        for j in range(i + 1, N):
            calc_vdw = True  # Assume we will calculate vdW for this pair
            # Exclude bonded pairs
            for bond in bonds_new:
                if i in bond and j in bond:
                    calc_vdw = False
                    break
            # Exclude pairs involved in angles
            if calc_vdw:
                for angle in angles_new:
                    if i in angle and j in angle:
                        calc_vdw = False
                        break

            # Retrieve atom types and coordinates
            atom1_type, x1, y1, z1 = atoms[i]
            atom2_type, x2, y2, z2 = atoms[j]
            r_ij = math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

            if calc_vdw:
                # Retrieve epsilon and sigma values
                epsilon_i = epsilon_values.get(atom1_type)
                epsilon_j = epsilon_values.get(atom2_type)
                sigma_i = sigma_values.get(atom1_type)
                sigma_j = sigma_values.get(atom2_type)

                if epsilon_i is None or epsilon_j is None:
                    raise ValueError(f"Unknown atom type: {atom1_type} or {atom2_type}")

                # Compute mixed epsilon and sigma using geometric mean
                epsilon_ij = math.sqrt(epsilon_i * epsilon_j)
                sigma_ij = 2*math.sqrt(sigma_i * sigma_j)

                # Compute Lennard-Jones potential
                term12 = (sigma_ij / r_ij) ** 12
                term6 = (sigma_ij / r_ij) ** 6
                energy_lj = 4 * epsilon_ij * (term12 - term6)

                # Accumulate the total energy
                total_energy += energy_lj
                energy = energy_lj
            else:
                # For uncalculated pairs, energy is zero
                energy = 0.0

            # Print the current pair details
            print(f"{atom1_type} {i + 1:<3}- {atom2_type} {j + 1:<3}: {r_ij:>15.4f} {energy:>20.4f}")

    print(f"\nTotal van der Waals Energy: {total_energy:.4f} kcal/mol")


# Example Input


# Calculate and print van der Waals energy
calculate_and_print_vdw_energy(atoms, bonds_new, angles_new)


Atom Pair          Distance (Å)    Energy (kcal/mol)
C 1  - C 2  :          1.5536               0.0000
C 1  - H 3  :          1.0815               0.0000
C 1  - H 4  :          1.0684               0.0000
C 1  - H 5  :          1.2502               0.0000
C 1  - H 6  :          2.1402               0.0000
C 1  - H 7  :          2.1532               0.0000
C 1  - H 8  :          2.2179               0.0000
C 2  - H 3  :          2.2093               0.0000
C 2  - H 4  :          2.1798               0.0000
C 2  - H 5  :          2.3082               0.0000
C 2  - H 6  :          1.0176               0.0000
C 2  - H 7  :          1.1018               0.0000
C 2  - H 8  :          1.1384               0.0000
H 3  - H 4  :          1.7824               0.0000
H 3  - H 5  :          1.6785               0.0000
H 3  - H 6  :          2.5092              -0.0215
H 3  - H 7  :          2.5543              -0.0258
H 3  - H 8  :          3.1375              -0.0192
H 4  - H 5  :          1.9853

# New Section